# Деревья

Рассмотрим вопрос как на самом деле происходит сплит по непрерывной фиче в дереве

Не будем останавливаться на всех возможных реализациях обучения деревьев

*   ID3
*   CART (с модификациями используется в sklearn)
*   C4.5 (J48)
*   C5.0
*   CN2
*   CHAID
*   Symmetric/Oblivious Trees (catboost)

Рассмотрим только вопрос сплита




Давайте напишем пару вспомогательных функций

In [4]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from operator import itemgetter

In [5]:
VERY_BIG = 1e90

In [12]:
def split_criterion(x, x_treshold, target, type = 'entropy'):
  assert len(x) == len(target)
  if type not in ['MSE', 'entropy', 'gini_impurity']:
    return 'incorrect criterion type'
  x = pd.Series(x)
  target = pd.Series(target, index = x.index)
  target_left, target_right = target[x < x_treshold], target[x >= x_treshold]
  if (not len(target_left)) or (not len(target_right)):
    return VERY_BIG
  if type == 'MSE':
    x_left, x_right = x[x < x_treshold], x[x >= x_treshold]
    prediction_left, prediction_right = np.mean(x_left), np.mean(x_right)
    mse_left, mse_right = sum((target_left - prediction_left) ** 2), sum((target_right - prediction_right) ** 2)
    return (len(target_left) * mse_left + len(target_right) * mse_right) / len(target)

  p_left, p_right = np.mean(target_left), np.mean(target_right)
  if type == 'gini_impurity':
    gini_left = 1 - (p_left**2 + (1-p_left)**2)
    gini_right = 1 - (p_right**2 + (1-p_right)**2)
    return (len(target_left)*gini_left + len(target_right)*gini_right) / len(target)
  if type == 'entropy':
    if (not p_left) or (not p_right):
      return VERY_BIG
    else:
      return -p_left * np.log2(p_left) -p_right * np.log2(p_right)

 **ОПРЕДЕЛЕНИЕ GINI IMPURITY**

Для бинарной классификации с вероятностью p класса 1:

**Gini(p) = 1 - p² - (1-p)² = 2p(1-p)**

---

 **ОБОСНОВАНИЕ ВЗВЕШЕННОГО СРЕДНЕГО**

**Исходное состояние:**
- Родительский узел: N samples  
- Gini родителя: Gini_parent

**После разделения:**
- Левая группа: N_L samples, Gini = Gini_L
- Правая группа: N_R samples, Gini = Gini_R

**Общая неопределенность после разделения:**
**Gini_after = (N_L/N) × Gini_L + (N_R/N) × Gini_R**

---

 **INFORMATION GAIN (целевой показатель)**

**Information Gain = Gini_before - Gini_after**

где:
**Gini_after = (N_L × Gini_L + N_R × Gini_R) / N**

---

**ПОЧЕМУ ИСХОДНЫЙ ПОДХОД НЕВЕРЕН**

Исходный код использовал: **1 - (p_L² + p_R²)**

**ПРОБЛЕМЫ:**
1. Складывает вероятности из разных групп
2.  Игнорирует размеры групп N_L и N_R  
3. Не сравнивает с исходной неопределенностью

---

 **ПРАВИЛЬНЫЙ МАТЕМАТИЧЕСКИЙ ПОДХОД**

**Для каждой группы:**
Gini_L = 2 × p_L × (1-p_L)
Gini_R = 2 × p_R × (1-p_R)

**Взвешенное среднее:**
**Gini_after = (N_L × Gini_L + N_R × Gini_R) / N**

**Information Gain:**
**IG = Gini_parent - Gini_after**

---

 **ИНТУИТИВНОЕ ОБЪЯСНЕНИЕ**

Большая группа с низкой неопределенностью должна вносить **БОЛЬШИЙ ВКЛАД** в общую оценку, чем маленькая группа с высокой неопределенностью.

 **ВЗВЕШЕННОЕ СРЕДНЕЕ ОБЕСПЕЧИВАЕТ ЭТУ ПРОПОРЦИОНАЛЬНОСТЬ!**

Давайте посмотрим на выбор сплита перебором порога в задаче классификации

In [13]:
feature = [5, 6, 7, 8, 9, 10, 2]
target = [1, 0, 1, 0, 1, 1, 1]
t = []
for feature_treshold in feature[0:-1]:
  t.append([feature_treshold, split_criterion(feature, feature_treshold, target, type = 'entropy')])
t = sorted(t, reverse = True, key = itemgetter(1))
print(tabulate(t, headers = ['Порог', 'Значение критерия']))

  Порог    Значение критерия
-------  -------------------
      7             0.701253
      8             0.701253
      6             0.442179
      9             0.442179
      5             0.389975
     10             0.389975


In [14]:

feature = [5, 6, 7, 8, 9, 10, 2]
target = [1, 0, 1, 0, 1, 1, 1]

sorted_indices = np.argsort(feature)
feature_sorted = [feature[i] for i in sorted_indices]
target_sorted = [target[i] for i in sorted_indices]

print("Отсортированные данные:")
print("Feature:", feature_sorted)
print("Target: ", target_sorted)
print()

t = []
for i in range(len(feature_sorted) - 1):
    feature_treshold = (feature_sorted[i] + feature_sorted[i + 1]) / 2  # среднее между соседями

    criterion_value = split_criterion(feature_sorted, feature_treshold, target_sorted, type='entropy')
    t.append([feature_treshold, criterion_value])

t = sorted(t, reverse=True, key=itemgetter(1))
print(tabulate(t, headers=['Порог', 'Значение критерия']))

Отсортированные данные:
Feature: [2, 5, 6, 7, 8, 9, 10]
Target:  [1, 1, 0, 1, 0, 1, 1]

  Порог    Значение критерия
-------  -------------------
    6.5             0.701253
    7.5             0.701253
    5.5             0.442179
    8.5             0.442179
    3.5             0.389975
    9.5             0.389975


Давайте убедимся что пороги 3.5 и 9.5 одинаково хорошо разбивают дерево, посчитаем для этого частоту единичек в поддеревьях

In [15]:
feature = pd.Series(feature)
target = pd.Series(target, index = feature.index)
print(np.mean(target[feature < 3.5]), np.mean(target[feature >= 3.5]))
print(np.mean(target[feature < 9.5]), np.mean(target[feature >= 9.5]))

1.0 0.6666666666666666
0.6666666666666666 1.0


Теперь перейдем непосредственно к стратегиям сплитов

In [16]:
from sklearn.datasets import make_classification

In [17]:
random_state = 88
x, y = make_classification(5_000, 10, random_state = random_state)

Теперь реализуем самую простую стратегию выбора первого узла дерева

In [18]:
%%time
best_splits = []
for idx_feature, feature in enumerate(x.T):
  t = []
  for feature_treshold in feature[0:-1]:
    t.append([feature_treshold, split_criterion(feature, feature_treshold, y, type = 'entropy')])
  best_split_index = np.nanargmin(np.array(t)[:, 1], axis = 0)
  best_splits.append([idx_feature, t[best_split_index][0], t[best_split_index][1]])
best_splits = sorted(best_splits, reverse = True, key = itemgetter(1))
print(tabulate(best_splits, headers = ['Feature', 'Порог', 'Значение критерия']))
print('the winner is')
best_of_best_split = np.nanargmin(np.array(best_splits)[:,2], axis = 0)
print(tabulate([best_splits[best_of_best_split]], headers = ['Feature', 'Порог', 'Значение критерия']))

  Feature     Порог    Значение критерия
---------  --------  -------------------
        1   4.34112             0.499956
        9   4.18103             0.499956
        6   3.51631             0.499956
        2   3.46727             0.499956
        5   3.24306             0.964164
        4   2.29289             0.499956
        7  -1.10045             0.468756
        8  -3.53499             0.499956
        3  -3.54405             0.499956
        0  -3.85811             0.499956
the winner is
  Feature     Порог    Значение критерия
---------  --------  -------------------
        7  -1.10045             0.468756
CPU times: user 27.3 s, sys: 25.7 ms, total: 27.3 s
Wall time: 27.8 s


Долго считалось!!

Мы перебрали все фичи и все пороги -> всего около 5 000 сплитов

Как подсчитать радикально быстрее?

Через гистограммы, но это будет другое дерево, точность упадет

Подсказка: используйте функцию pd.qcut

In [21]:
%%time
best_splits = []
for idx_feature, feature in enumerate(x.T):
    t = []
    _, bins = np.histogram(feature, bins=128)
    #bins = np.percentile(feature, np.linspace(0, 100, 129))
    for feature_treshold in bins[:-1]:
        criterion_value = split_criterion(feature, feature_treshold, y, type='entropy')
        t.append([feature_treshold, criterion_value])
    best_split_index = np.nanargmin(np.array(t)[:, 1])
    best_splits.append([idx_feature, t[best_split_index][0], t[best_split_index][1]])
best_splits = sorted(best_splits, reverse=True, key=itemgetter(2))
print(tabulate(best_splits, headers=['Feature', 'Порог', 'Значение критерия']))
print('the winner is')
best_of_best_split = np.nanargmin(np.array(best_splits)[:,2], axis=0)
print(tabulate([best_splits[best_of_best_split]], headers=['Feature', 'Порог', 'Значение критерия']))

  Feature     Порог    Значение критерия
---------  --------  -------------------
        8  -1.01648             0.990269
        5   2.08983             0.98471
        3   1.76306             0.974604
        2   2.45732             0.974462
        9   2.37604             0.959239
        6   2.37214             0.942447
        4  -1.29534             0.873724
        1   3.04094             0.860823
        0  -2.3691              0.524808
        7  -1.10756             0.470641
the winner is
  Feature     Порог    Значение критерия
---------  --------  -------------------
        7  -1.10756             0.470641
CPU times: user 1.08 s, sys: 1.91 ms, total: 1.08 s
Wall time: 1.19 s


**Задание**: при числе бинов в гистограмме равной 128 какая фича окажется в корне дерева?



1.   feature_0
2.   feature_1
3.   feature_2
4.   feature_3
5.   feature_4
6.   feature_5
7.   feature_6
#8.   feature_7
9.   feature_8
10.  feature_9

Ответ 7 фича в корне дерева

Стратегии выбора порогов очень разные, например в catboost:

https://catboost.ai/docs/en/concepts/quantization

# Градиентный бустинг



Мы же с вами здесь реализуем базовый кейс -- градиентный бустинг в задаче регресии


In [20]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

In [22]:
class simple_gradient_boosting_regressor():
    def __init__(self, n_estimators=20, max_depth=5, learning_rate=0.2, random_state=55):
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.n_estimators = n_estimators
        self.trees = []
        self.initial_pred = None

    def fit(self, x, y, verbose=True):
        preds = np.full_like(y, np.mean(y))
        self.initial_pred = np.mean(y)

        for idx in range(self.n_estimators):
            residuals = y - preds

            current_tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                random_state=self.random_state
            )
            current_tree.fit(x, residuals)

            self.trees.append(current_tree)

            preds += self.learning_rate * current_tree.predict(x)

            if verbose:
                mse = mean_squared_error(y, preds)
                print(f'MSE after {idx+1} tree is {mse:.3f}')

    def predict(self, x):
        preds = np.full(x.shape[0], self.initial_pred)

        for tree in self.trees:
            preds += self.learning_rate * tree.predict(x)

        return preds

In [23]:
random_state = 55
x, y = make_regression(1000, 5, random_state = random_state)

In [24]:
gbr = simple_gradient_boosting_regressor()
gbr.fit(x, y)

MSE after 1 tree is 11217.737
MSE after 2 tree is 7910.404
MSE after 3 tree is 5650.280
MSE after 4 tree is 4118.589
MSE after 5 tree is 3032.511
MSE after 6 tree is 2233.586
MSE after 7 tree is 1676.377
MSE after 8 tree is 1283.969
MSE after 9 tree is 984.848
MSE after 10 tree is 774.297
MSE after 11 tree is 603.629
MSE after 12 tree is 483.340
MSE after 13 tree is 392.615
MSE after 14 tree is 321.182
MSE after 15 tree is 267.361
MSE after 16 tree is 226.771
MSE after 17 tree is 193.463
MSE after 18 tree is 169.164
MSE after 19 tree is 149.315
MSE after 20 tree is 132.892


11217.737 / 132.892 ≈ 84.4 раза


**Задание:**
Во сколько раз упала ошибка MSE на трейне при параметрах, указанных в коде?



1.   Не упала
2.   Упала менее чем в 2 раза
3.   Упала от 2х до 10 раз
#4.   Упала от 10 до 100 раз
5.   Упала от ста до тысячи раз
6.   Упала более чем в тысячу раз

Ответ номер 4